In [1]:
import psycopg2
import csv
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

In [2]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )


In [3]:
def generate_supplier_orders(n=1000):
    conn = get_connection()
    cursor = conn.cursor()
    
    cursor.execute("SELECT supplier_id FROM suppliers")
    suppliers = [supplier[0] for supplier in cursor.fetchall()]
    
    cursor.execute("SELECT product_id, buyin_price FROM products")
    products = cursor.fetchall()
    
    if len(products) < n:
        print(f"Error: Not enough unique products. Need at least {n} unique products.")
        cursor.close()
        conn.close()
        return
    
    random.shuffle(products)

    orders_data = []

    for i in range(n):
        supplier_id = random.choice(suppliers)
        product_id, price_per_unit = products[i]
        quantity = random.randint(50, 200)
        total_price = price_per_unit * quantity
        order_date = datetime.now() - timedelta(days=random.randint(0, 365))
        orders_data.append((supplier_id, product_id, quantity, total_price, order_date.strftime('%Y-%m-%d')))
        
        cursor.execute(
            "INSERT INTO supplier_order (supplier_id, product_id, quantity, price, date) VALUES (%s, %s, %s, %s, %s)",
            (supplier_id, product_id, quantity, total_price, order_date)
        )

    conn.commit()
    cursor.close()
    conn.close()
    # Write the generated ratings data to a CSV file
    with open('C:/Users/Yihua/Desktop/5310 project/supplier_orders.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['supplier_id', 'product_id', 'quantity', 'price', 'date'])
        csv_writer.writerows(orders_data)

    print(f"Generated {len(orders_data)} rows of supplier orders data.")

if __name__ == "__main__":
    generate_supplier_orders()

Generated 1000 rows of supplier orders data.
